In [1]:
# Load necessary modules

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
# Data preparation

num=9
fn = ['DRB동일.csv', 'KEC.csv', 'SK아이이테크놀로지.csv', '기신정기.csv', '동양피스톤.csv', '두산퓨얼셀.csv', '디와이파워.csv', '삼아알미늄.csv', '티와이홀딩스.csv', '화승코퍼레이션.csv']

df = pd.read_csv('data/'+fn[num], index_col = 'Date', parse_dates=True)
df['Mid']=(df['Low']+df['High'])/2

cut_line = len(df['Mid'])-round(len(df['Mid'])/10)

train_data = df['Mid'].values[:cut_line].reshape(-1,1).astype('float')
test_data = df['Mid'].values[cut_line:].reshape(-1,1).astype('float')


# Scale data

from sklearn.preprocessing import minmax_scale as mm
train_data = mm(train_data)
test_data = mm(test_data)

In [3]:
def create_seq(data, seq_length):
    x = []
    y = []
    for i in range(len(data)-seq_length):
        x.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    x = np.array(x)
    y = np.array(y)
    return x, y

In [4]:
class LSTM1(nn.Module):
    
    def __init__(self, input_size, hidden_size, num_layers, seq_len):
        super(LSTM1, self).__init__()
        self.num_layers = num_layers #number of layers 
        self.input_size = input_size #input size (number of features)
        self.hidden_size = hidden_size #hidden state 
        self.seq_len = seq_len #sequence length 
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers) #lstm 
        self.fc = nn.Linear(self.hidden_size, 1) #fully connected last layer 
    
    def init_hidden(self):
        self.hidden = (
            torch.zeros(self.num_layers, self.seq_len, self.hidden_size),
            torch.zeros(self.num_layers, self.seq_len, self.hidden_size)          
                      )
    
    def forward(self,seqs):
        lstm_out, self.hidden = self.lstm(seqs.view(len(seqs), self.seq_len, -1), self.hidden) #lstm with input, hidden, and internal state 
        lstm_last = lstm_out.view(self.seq_len, len(seqs), self.hidden_size)[-1]
        out = self.fc(lstm_last)
        return out

In [5]:
# Train

def train(num_epochs, learning_rate, input_size, hidden_size, num_layers, seq_len):
    x_train, y_train = create_seq(train_data, seq_len)
    x_test, y_test = create_seq(test_data, seq_len)
    x_train, y_train, x_test, y_test = map(lambda data:torch.from_numpy(data).float(), [x_train, y_train, x_test, y_test])

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # device
    lstm1 = LSTM1(input_size, hidden_size, num_layers, seq_len).to(device)

    loss_function = torch.nn.MSELoss()    # mean-squared error for regression
    optimizer = torch.optim.Adam(lstm1.parameters(), lr=learning_rate)  # adam optimizer

    for epoch in range(num_epochs):
        lstm1.init_hidden()
        y_pred = lstm1(x_train)  
        loss = loss_function(y_pred, y_train) 
        # obtain the loss function 
        optimizer.zero_grad() 
        loss.backward() #calculates the loss of the loss function 
        optimizer.step() #improve from loss, i.e backprop 
        if epoch % 100 == 0: 
            print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))

            
    # Testing the model 
            
    # plt.rc('font', family='NanumGothic') # use when korean text output is needed
    data_predict = lstm1(x_test.to(device)).data.detach().cpu().numpy()
    history_cut = 0 # length of latest history to view. Set to 0 to see whole test set.
    plt.figure(figsize=(7.5,4.5)) #plotting
    plt.plot(y_test[-history_cut:], label='actual price', marker='o') #actual plot
    plt.plot(data_predict[-history_cut:], label='predicted price', marker='v') #predicted plot
    plt.title(f"Company {num}")
    plt.legend(loc='upper left')

    
    # Save model performance and hyperparameters
    
    filename = datetime.now().strftime("%Y%m%d-%H%M%S")
    plt.savefig(f'performance/{filename}.png')
    with open("performance/performance.txt",'a') as f:
        f.write(f"""
id:\t\t{filename}
num_epochs:\t{num_epochs}
learning_rate:\t{learning_rate}
hidden_size:\t{hidden_size}
num_layers:\t{num_layers}
seq_len:\t\t{seq_len}
loss:\t\t{round(loss.item(),5)}
""")

In [6]:
# Hyperparameters 

num_epochs = 5000 
learning_rate = 0.001

input_size = 1 #number of features
hidden_sizes = [1,3,5,10,20] #number of features in hidden state
num_layerss = [1,2,3] #number of stacked lstm layers
seq_lens = [1,5,10]


# Run training with differing hyperparameters

for hidden_size in hidden_sizes:
    for seq_len in seq_lens:
        for num_layers in num_layerss:
            train(num_epochs, learning_rate, input_size, hidden_size, num_layers, seq_len)

Epoch: 0, loss: 0.05255


KeyboardInterrupt: 